In [12]:
import json
import os

def read_data(datafile=os.path.join(os.getcwd(), '.data', 'master.json')):
    with open(datafile, mode='r') as f:
        lines = [json.loads(l) for l in f.readlines()]
        return lines

data = read_data()

print(data[:10])

[{'feed_name': 'The Economist - Science and Technology', 'img_loc': 'http://cdn.static-economist.com/sites/default/files/images/print-edition/20180217_STD001_0.jpg', 'pub_datetime': '2018-02-15 15:48:17', 'summary': 'science fiction is littered with examples of intelligent computers, from hal 9000 in 2001: a space odyssey to eddie in the hitchhikers guide to the galaxy. one thing such fictional machines have in common is a tendency to go wrong, to the detriment of the characters in the story. hal murders most of the crew of a mission to jupiter. eddie obsesses about trivia, and thus puts the spacecraft he is in charge of in danger of destruction. in both cases, an attempt to build something useful and helpful has created a monster.  successful science fiction necessarily plays on real hopes and fears. in the 1960s and 1970s, when hal and eddie were dreamed up, attempts to create artificial intelligence (ai) were floundering, so both hope and fear were hypothetical. but that has changed

In [18]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly import graph_objs as go
from collections import Counter
init_notebook_mode(connected=True)

kwds = []
[kwds.extend(d['kwds']) for d in data]

pltdata = [go.Histogram(x=kwds)]
plot(pltdata)




'file:///home/cmustard/Projects/spyndicated/spyndicated/temp-plot.html'

In [20]:
import numpy as np
from gensim import corpora, models

texts = [d['kwds'] for d in data]
dictionary = corpora.Dictionary(texts)
lda = models.ldamodel.LdaModel(corpus=dictionary)



TypeError: 'int' object is not iterable